In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd 


In [24]:
def get_jobs(keyword,location="", num_jobs=5, verbose=False):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    # options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    # driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1120, 1000)
   
    url = 'https://www.glassdoor.com/Job/jobs.htm'
    driver.get(url)

    search_box = driver.find_element(By.XPATH,'//*[@id="sc.keyword"]')
    search_box.send_keys(keyword)
    if location=="":
        search_button = driver.find_element(By.XPATH,'//*[@data-test="search-bar-submit"]')
        search_button.click()
    else:
        location_box = driver.find_element(By.XPATH,'//*[@id="sc.location"]')
        location_box.click()
        driver.find_element(By.XPATH,'//*[@data-test="search-bar-location-input-clear"]').click()
        location_box.send_keys(location)
        search_button = driver.find_element(By.XPATH,'//*[@data-test="search-bar-submit"]')
        search_button.click()


# Click the search button
    

    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.XPATH, '//*[@data-test="job-tile"]')))
    See_button=driver.find_element(By.XPATH,'//*[@data-test="jobs-location-see-all-link"]')
    See_button.click()
    
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        
        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_element_located((By.XPATH, '//*[@data-test="jobListing"]')))


        #Test for the "Sign Up" prompt and get rid of it.
        try:
            button = driver.find_element(By.XPATH,"//*[@data-selected='true']")
            button.click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element(By.XPATH,"//*[@class='SVGInline-svg modal_closeIcon-svg']").click()  #clicking to the X.
        except NoSuchElementException:
            pass

     
        
        #Going through each job in this page
        job_buttons = driver.find_elements(By.XPATH,"//*[@data-test='jobListing']")  #jl for Job Listing. These are the buttons we're going to click.
        
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                print("Completed")
                break

            job_button.click()  #You might 
            time.sleep(2)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element(By.XPATH,'.//div[@data-test="employerName"]').text
                    company_name = company_name.split("\n", 1)[0]
                    location = driver.find_element(By.XPATH,'.//div[@data-test="location"]').text
                    job_title = driver.find_element(By.XPATH,'.//div[@data-test="jobTitle"]').text
                    # job_description = driver.find_element(By.XPATH,'.//button[@class="minor descSnippet css-1xq5z8r eigr9kq1"]//span').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element(By.XPATH,'.//li[@data-selected="true"]//div//div//div//span[@data-test="detailSalary"]').text
                salary_estimate = salary_estimate.split("(", 1)[0]

            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            
            try:
                rating = driver.find_element(By.XPATH,'.//span[@data-test="detailRating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                # print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))


            try:
                driver.find_element(By.XPATH,'.//div[@id="CompanyContainer"]')
                try:
                    size = driver.find_element(By.XPATH,'.//div[@id="EmpBasicInfo"]//div//div//div//span[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1


                try:
                    founded = driver.find_element(By.XPATH,'.//div[@id="EmpBasicInfo"]//div//div//div//span[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element(By.XPATH,'.//div[@id="EmpBasicInfo"]//div//div//div//span[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element(By.XPATH,'.//div[@id="EmpBasicInfo"]//div//div//div//span[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element(By.XPATH,'.//div[@id="EmpBasicInfo"]//div//div//div//span[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element(By.XPATH,'.//div[@id="EmpBasicInfo"]//div//div//div//span[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                # try:
                #     competitors = driver.find_element(By.XPATH,'.//div[@id="EmpBasicInfo"]//div//div//div//span[text()="Headquaters"]//following-sibling::*').text
                # except NoSuchElementException:
                #     competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                # print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                # print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            # "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue
            })
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element(By.XPATH,'.//button[@data-test="pagination-next"]').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [25]:
path = "D:/shreeraj/Coding/Data\ science/Project1/ds_salary_proj"

df = get_jobs("QA testing","california",7)

df

Progress: 0/7
Progress: 1/7
Progress: 2/7
Progress: 3/7
Progress: 4/7
Progress: 5/7
Progress: 6/7
Progress: 7/7
Completed


,Job Title,Salary Estimate,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Trainee - QA,$48K - $77K,4.0,LeewayHertz,"San Francisco, CA",51 to 200 Employees,-1,Company - Private,Computer Hardware Development,Information Technology,Unknown / Non-Applicable
1,QA Engineer - Section 508/Accessibility testing,$40.00 - $60.98 Per Hour,-1,TAA Solutions LLC,"San Francisco, CA",1 to 50 Employees,-1,Company - Public,-1,-1,Unknown / Non-Applicable
2,QA Tester,$55.00 Per Hour,3.4,VTech,"San Bernardino, CA",Unknown,-1,Company - Public,Electronics Manufacturing,Manufacturing,$1 to $5 billion (USD)
3,Junior QA Manual Tester,$70K - $105K,3.7,Embtel Inc,"Burlingame, CA",51 to 200 Employees,2011,Company - Private,Information Technology Support Services,Information Technology,$5 to $25 million (USD)
4,QA Engineers,$61K - $100K,4.0,Leothundra,"La Palma, CA",1 to 50 Employees,2012,Company - Private,Information Technology Support Services,Information Technology,Less than $1 million (USD)
5,Software QA Engineer (Manual Tester),-1,4.1,Sun-Net Inc.,"San Jose, CA",1 to 50 Employees,2000,Company - Private,Enterprise Software & Network Solutions,Information Technology,Unknown / Non-Applicable
6,SDET/QA Automation Engineer,$50.00 - $55.00 Per Hour,4.2,Validation Associates LLC,"San Diego, CA",51 to 200 Employees,2003,Company - Private,HR Consulting,Human Resources & Staffing,Unknown / Non-Applicable
